In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from pylab import rcParams
from keras.models import Model, load_model
#from keras.layers import Input, Dense
from tensorflow.keras.layers import Input, Dense
from tensorflow import keras
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
df = pd.read_csv('./DataP2/HDDData_Q3.csv')

In [3]:
print(df.shape)
df.head()

(6443112, 95)


,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2016-07-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,135.0,108.0,127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-07-01,MJ0351YNG9WJSA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-07-01,PL1321LAG34XWH,Hitachi HDS5C4040ALE630,1.976651e-311,0,100,0,134.0,101.0,130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-01,Z305B2QN,ST4000DM000,1.976651e-311,0,117,144377848,NaN,NaN,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
"""le = LabelEncoder()
encoded = le.fit_transform(df['model'])
le.inverse_transform(encoded)"""

array(['Hitachi HDS5C3030ALA630', 'Hitachi HDS5C3030ALA630',
       'Hitachi HDS5C4040ALE630', ..., 'Hitachi HDS5C3030ALA630',
       'ST4000DM000', 'ST4000DM000'], dtype=object)

In [4]:
#df.mean()
df.columns

Index(['date', 'serial_number', 'model', 'capacity_bytes', 'failure',
       'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized',
       'smart_2_raw', 'smart_3_normalized', 'smart_3_raw',
       'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized',
       'smart_5_raw', 'smart_7_normalized', 'smart_7_raw',
       'smart_8_normalized', 'smart_8_raw', 'smart_9_normalized',
       'smart_9_raw', 'smart_10_normalized', 'smart_10_raw',
       'smart_11_normalized', 'smart_11_raw', 'smart_12_normalized',
       'smart_12_raw', 'smart_13_normalized', 'smart_13_raw',
       'smart_15_normalized', 'smart_15_raw', 'smart_22_normalized',
       'smart_22_raw', 'smart_183_normalized', 'smart_183_raw',
       'smart_184_normalized', 'smart_184_raw', 'smart_187_normalized',
       'smart_187_raw', 'smart_188_normalized', 'smart_188_raw',
       'smart_189_normalized', 'smart_189_raw', 'smart_190_normalized',
       'smart_190_raw', 'smart_191_normalized', 'smart_191_raw',
       'smart_1

In [5]:
# Constants
RANDOM_SEED = 314
TEST_PCT = 0.2

In [6]:
df_norm = df

# Drop the following columns
df_norm.drop(['date', 'model', 'serial_number'], axis=1, inplace=True)

# Drop normalized columns
df_norm = df_norm[df_norm.columns.drop(list(df_norm.filter(regex='normalized')))]

# Drop Duplicate Rows
df_norm = df_norm.drop_duplicates(subset=None, keep='first', inplace=False).reset_index(drop=True)

# Drop Columns where all entries are NAN
df_norm.dropna(axis=1, how='all', inplace=True)

# Replace NaNs with mean
df_norm.fillna(df_norm.mean(), inplace=True)

df_norm.head()
df_norm.shape

(6330275, 43)

In [7]:
# Split
train_x, test_x = train_test_split(df_norm, test_size=TEST_PCT, random_state=RANDOM_SEED)

# Transform to DF
trainDf = pd.DataFrame(train_x)
trainDf.columns = df_norm.columns

testDf = pd.DataFrame(test_x)
testDf.columns = df_norm.columns

# Training Data
train_x = trainDf[trainDf.failure == 0] # Healthy drives
train_x = trainDf.drop(['failure'], axis=1) # drop the Failure column

# Testing Data
test_y = testDf['failure']
test_x = testDf.drop(['failure'], axis=1) #drop the failure column

display(train_x.head())
display(test_x.head())
display(test_y.head())

# Scale and Standardize & Transform DF to ndarray
train_x = StandardScaler().fit_transform(train_x)
test_x = StandardScaler().fit_transform(test_x)
test_y = test_y.values

print(train_x)
print(test_x)

,capacity_bytes,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,smart_10_raw,...,smart_224_raw,smart_225_raw,smart_226_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw
3043181,1.482490e-311,0,0.000000,561,18,0,0,0.000000,42744,0,...,0.0,133515.515312,340.604588,1.983036e+13,2.428052e+10,7.977672e+10,1.263392e+07,45813.830131,0.0,0.0
3227557,3.953298e-311,35035056,85.020509,0,2,0,260964555,33.647374,2006,0,...,0.0,133515.515312,340.604588,2.497395e+14,1.958744e+10,4.323660e+09,1.263392e+07,45813.830131,0.0,0.0
434626,1.976651e-311,0,102.000000,564,37,0,0,43.000000,27669,0,...,0.0,133515.515312,340.604588,1.983036e+13,2.428052e+10,7.977672e+10,1.263392e+07,45813.830131,0.0,0.0
4378914,1.976651e-311,0,104.000000,437,6,0,0,42.000000,20816,0,...,0.0,133515.515312,340.604588,1.983036e+13,2.428052e+10,7.977672e+10,1.263392e+07,45813.830131,0.0,0.0
3960721,1.976651e-311,165899856,85.020509,0,15,0,412923988,33.647374,21894,0,...,0.0,133515.515312,340.604588,2.177200e+04,2.444987e+10,2.063105e+11,1.263392e+07,45813.830131,0.0,0.0


,capacity_bytes,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,smart_10_raw,...,smart_224_raw,smart_225_raw,smart_226_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw
3591355,1.482490e-311,0,101.000000,570,16,0,0,32.000000,38209,0,...,0.0,133515.515312,340.604588,1.983036e+13,2.428052e+10,7.977672e+10,1.263392e+07,45813.830131,0.0,0.0
6261830,1.976651e-311,218181792,85.020509,0,2,0,397334992,33.647374,9664,0,...,0.0,133515.515312,340.604588,9.549000e+03,3.319973e+10,7.209512e+09,1.263392e+07,45813.830131,0.0,0.0
2117812,1.976651e-311,117848624,85.020509,0,4,0,301619647,33.647374,7184,0,...,0.0,133515.515312,340.604588,7.098000e+03,2.811389e+10,4.748826e+09,1.263392e+07,45813.830131,0.0,0.0
4511696,3.953298e-311,82635184,85.020509,0,2,0,273307953,33.647374,2459,0,...,0.0,133515.515312,340.604588,1.118925e+14,1.892580e+10,4.869848e+09,1.263392e+07,45813.830131,0.0,0.0
5660442,1.976651e-311,231960256,85.020509,0,20,0,804953564,33.647374,26740,0,...,0.0,133515.515312,340.604588,2.661100e+04,2.718040e+10,1.438896e+11,1.263392e+07,45813.830131,0.0,0.0


3591355    0
6261830    0
2117812    0
4511696    0
5660442    0
Name: failure, dtype: int64

[[-5.34266709e-312 -8.94437078e-001 -3.61631809e+000 ... -8.43603424e-006
   0.00000000e+000  0.00000000e+000]
 [ 1.93654088e-311 -4.60049964e-001 -2.49431480e-006 ... -8.43603424e-006
   0.00000000e+000  0.00000000e+000]
 [-4.01051906e-313 -8.94437078e-001  7.22213831e-001 ... -8.43603424e-006
   0.00000000e+000  0.00000000e+000]
 ...
 [-4.01051906e-313  1.38423374e+000 -2.49431480e-006 ... -8.43603424e-006
   0.00000000e+000  0.00000000e+000]
 [ 1.93654088e-311  7.40213361e-001 -2.49431480e-006 ... -8.43603424e-006
   0.00000000e+000  0.00000000e+000]
 [-4.01051906e-313  3.95900893e-001 -2.49431480e-006 ... -8.43603424e-006
   0.00000000e+000  0.00000000e+000]]
[[-5.33517447e-312 -8.93982491e-001  6.79065486e-001 ...  3.63846426e-005
   0.00000000e+000  0.00000000e+000]
 [-3.93559285e-313  1.81148111e+000  9.96806732e-006 ...  3.63846426e-005
   0.00000000e+000  0.00000000e+000]
 [-3.93559285e-313  5.67345514e-001  9.96806732e-006 ...  3.63846426e-005
   0.00000000e+000  0.00000000e+

In [8]:
train_x.shape[1]

42

In [14]:
# AutoEncoder

# Parameters
nb_epoch = 100
batch_size = 128000
input_dim = train_x.shape[1] #num of columns, 30
encoding_dim = 20 # 14
hidden_dim = int(encoding_dim / 2) #i.e. 7
learning_rate = 1e-7

# Encoder
input_layer = keras.Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)

# Decoder
decoder = Dense(hidden_dim, activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)

# Model
autoencoder = keras.Model(inputs=input_layer, outputs=decoder)

In [27]:
import os

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = keras.callbacks.ModelCheckpoint(filepath="autoencoder.h5",
                               save_best_only=True,
                               verbose=0)
logdir = os.path.join('logs')
tb = keras.callbacks.TensorBoard(log_dir=logdir,
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(train_x, train_x,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_x, test_x),
                    verbose=1, #callbacks=[cp, tb]
                    ).history

Train on 5064220 samples, validate on 1266055 samples
Epoch 1/100
5064220/5064220 [==============================] - 3s 1us/sample - loss: 0.8501 - accuracy: 0.0976 - val_loss: 0.8341 - val_accuracy: 0.1758
Epoch 2/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.8215 - accuracy: 0.2299 - val_loss: 0.8072 - val_accuracy: 0.2913
Epoch 3/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.7966 - accuracy: 0.3489 - val_loss: 0.7838 - val_accuracy: 0.4408
Epoch 4/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.7738 - accuracy: 0.4849 - val_loss: 0.7620 - val_accuracy: 0.5451
Epoch 5/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.7560 - accuracy: 0.5692 - val_loss: 0.7479 - val_accuracy: 0.5788
Epoch 6/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.7442 - accuracy: 0.5924 - val_loss: 0.7372 - val_accuracy: 0.5880
Epoch 7/100
5064220/5064220 

Epoch 54/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.5891 - accuracy: 0.7749 - val_loss: 0.5832 - val_accuracy: 0.7716
Epoch 55/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.5875 - accuracy: 0.7747 - val_loss: 0.5816 - val_accuracy: 0.7708
Epoch 56/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.5859 - accuracy: 0.7747 - val_loss: 0.5802 - val_accuracy: 0.7717
Epoch 57/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.5844 - accuracy: 0.7751 - val_loss: 0.5788 - val_accuracy: 0.7718
Epoch 58/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.5829 - accuracy: 0.7753 - val_loss: 0.5772 - val_accuracy: 0.7708
Epoch 59/100
5064220/5064220 [==============================] - 2s 0us/sample - loss: 0.5814 - accuracy: 0.7756 - val_loss: 0.5758 - val_accuracy: 0.7717
Epoch 60/100
5064220/5064220 [==============================] - 2s 0us/sampl

In [29]:
# serialize model to JSON
model_json = autoencoder.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
autoencoder.save_weights("autoencoder.h5")
print("Saved model to disk")

Saved model to disk


In [56]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("autoencoder.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

pred = loaded_model.predict(test_x)
mse = np.mean(np.power(test_x - pred, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': test_y})

Loaded model from disk


In [60]:
# Metrics
display(error_df.describe())
error_df

,Reconstruction_error,True_class
count,1.266055e+06,1.266055e+06
mean,5.289302e-01,5.134058e-05
std,4.011153e+01,7.165053e-03
min,6.885044e-03,0.000000e+00
25%,3.429383e-02,0.000000e+00
50%,5.219220e-02,0.000000e+00
75%,1.060162e-01,0.000000e+00
max,3.908855e+04,1.000000e+00


,Reconstruction_error,True_class
0,0.022707,0
1,0.019304,0
2,0.025056,0
3,0.072034,0
4,0.013155,0
...,...,...
1266050,0.133767,0
1266051,0.412972,0
1266052,0.015137,0
1266053,0.100162,0
